In [2]:
import os
import json
from datetime import datetime, timedelta

import dask
from dask.distributed import Client
from dask_ml.cluster import SpectralClustering
import geopandas as gpd
import hvplot.xarray
import matplotlib.pyplot as plt
import pyproj
from pystac_client import Client as pystac_client
import rasterio
import xarray as xr

from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo
from rasterio import features
from shapely.geometry import box, shape
from pyproj import Transformer


In [24]:
cl = Client()
cl

In [4]:
bbox = [24.849829673767093, 56.036640346352655, 24.882316589355472, 56.04725989803104]


geom = box(*bbox)  # minx, miny, maxx, maxy
gdf = gpd.GeoDataFrame({"name": ["bbox"]}, geometry=[geom], crs="EPSG:4326")
gdf.explore()  # requires `pip install geopandas[explore]`

In [5]:
catalog = pystac_client.open("https://earth-search.aws.element84.com/v1")
chunk={} # <-- use dask
time='2019-06-01/2020-09-30' # some summer months
res=10 # 10m resolution

query = catalog.search(
    collections=["sentinel-2-l2a"], datetime=time, limit=100,
    bbox=bbox, query={"eo:cloud_cover": {"lt": 0.1}}
)

items = list(query.items())
print(f"Found: {len(items):d} datasets")

Found: 7 datasets


In [6]:
item = items[2]
item

<Item id=S2A_35VLC_20191201_1_L2A>

In [7]:
grass_land = stac_load(
    [item],
    bands=(
        "coastal",
        "blue",
        "green",
        "red",
        "rededge1",
        "rededge2",
        "rededge3",
        "nir",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
    ),
    chunks=chunk,
    resolution=10,
    # crs="EPSG:4326",
    bbox=bbox,
)
grass_land


<xarray.Dataset> Size: 624kB
Dimensions:      (y: 125, x: 207, time: 1)
Coordinates:
  * y            (y) float64 1kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (x) float64 2kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
    spatial_ref  int32 4B 32635
  * time         (time) datetime64[ns] 8B 2019-12-01T09:44:50.584000
Data variables:
    coastal      (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    blue         (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    green        (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    red          (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    rededge1     (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    rededge2     (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    rededge3     (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    nir          (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    nir08        (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    nir09        (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    swir16       (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>
    swir22       (time, y, x) uint16 52kB dask.array<chunksize=(1, 125, 207), meta=np.ndarray>

In [8]:
grass_land_da = grass_land.isel(time=0).to_array(dim="band")
grass_land_da.attrs["crs"] = str(grass_land.odc.crs)
grass_land_da


<xarray.DataArray (band: 12, y: 125, x: 207)> Size: 621kB
dask.array<stack, shape=(12, 125, 207), dtype=uint16, chunksize=(1, 125, 207), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 1kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (x) float64 2kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
    spatial_ref  int32 4B 32635
    time         datetime64[ns] 8B 2019-12-01T09:44:50.584000
  * band         (band) object 96B 'coastal' 'blue' ... 'swir16' 'swir22'
Attributes:
    crs:      PROJCRS["WGS 84 / UTM zone 35N",BASEGEOGCRS["WGS 84",ENSEMBLE["...

In [9]:

rgb = (grass_land_da.sel(band=["red", "green", "blue"]) / 1000).clip(0, 1)

rgb_plot = rgb.hvplot.rgb(
    x="x", y="y", bands="band", title="Sentinel-2 RGB", tiles=True)


rgb_plot

:Overlay
   .Tiles.I :Tiles   [x,y]
   .RGB.I   :RGB   [x,y]   (R,G,B)

## Spectral clustering pipeline


In [10]:
flattened_xda = grass_land_da.stack(z=("y", "x"))
flattened_xda


<xarray.DataArray (band: 12, z: 25875)> Size: 621kB
dask.array<reshape, shape=(12, 25875), dtype=uint16, chunksize=(1, 25875), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref  int32 4B 32635
    time         datetime64[ns] 8B 2019-12-01T09:44:50.584000
  * band         (band) object 96B 'coastal' 'blue' ... 'swir16' 'swir22'
  * z            (z) object 207kB MultiIndex
  * y            (z) float64 207kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (z) float64 207kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
Attributes:
    crs:      PROJCRS["WGS 84 / UTM zone 35N",BASEGEOGCRS["WGS 84",ENSEMBLE["...

In [11]:
flattened_t_xda = flattened_xda.transpose("z", "band")
with xr.set_options(keep_attrs=True):
    rescaled_xda = ((flattened_t_xda - flattened_t_xda.mean()) / flattened_t_xda.std()).astype("float32")
rescaled_xda

<xarray.DataArray (z: 25875, band: 12)> Size: 1MB
dask.array<astype, shape=(25875, 12), dtype=float32, chunksize=(25875, 1), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref  int32 4B 32635
    time         datetime64[ns] 8B 2019-12-01T09:44:50.584000
  * band         (band) object 96B 'coastal' 'blue' ... 'swir16' 'swir22'
  * z            (z) object 207kB MultiIndex
  * y            (z) float64 207kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (z) float64 207kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
Attributes:
    crs:      PROJCRS["WGS 84 / UTM zone 35N",BASEGEOGCRS["WGS 84",ENSEMBLE["...

In [12]:
rescaled_xda = rescaled_xda.astype("float32")
X = cl.persist(rescaled_xda)
X

<xarray.DataArray (z: 25875, band: 12)> Size: 1MB
dask.array<astype, shape=(25875, 12), dtype=float32, chunksize=(25875, 1), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref  int32 4B 32635
    time         datetime64[ns] 8B 2019-12-01T09:44:50.584000
  * band         (band) object 96B 'coastal' 'blue' ... 'swir16' 'swir22'
  * z            (z) object 207kB MultiIndex
  * y            (z) float64 207kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (z) float64 207kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
Attributes:
    crs:      PROJCRS["WGS 84 / UTM zone 35N",BASEGEOGCRS["WGS 84",ENSEMBLE["...

In [13]:
clf = SpectralClustering(
    n_clusters=3,
    random_state=0,
    gamma=None,
    kmeans_params={"init_max_iter": 6},
    # persist_embedding=True,
)
clf


SpectralClustering(gamma=None, kmeans_params={'init_max_iter': 6}, n_clusters=3,
                   random_state=0)

In [14]:
%time clf.fit(X)


/opt/miniconda3/envs/pangeo/lib/python3.13/site-packages/dask/base.py:1105: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


CPU times: user 18.4 s, sys: 1.9 s, total: 20.3 s
Wall time: 34.1 s


SpectralClustering(gamma=None, kmeans_params={'init_max_iter': 6}, n_clusters=3,
                   random_state=0)

In [15]:
labels = clf.assign_labels_.labels_.compute()
labels.shape


(25875,)

In [16]:
template = flattened_t_xda[:, 0]
output_array = template.copy(data=labels)
clusters = output_array.unstack()
clusters.attrs["crs"] = grass_land_da.attrs.get("crs")
clusters


<xarray.DataArray (y: 125, x: 207)> Size: 104kB
array([[2, 2, 2, ..., 2, 0, 0],
       [0, 2, 2, ..., 0, 0, 0],
       [0, 2, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 2, 2],
       [0, 0, 0, ..., 2, 2, 2],
       [0, 0, 0, ..., 2, 2, 2]], shape=(125, 207), dtype=int32)
Coordinates:
  * y            (y) float64 1kB 6.213e+06 6.213e+06 ... 6.212e+06 6.212e+06
  * x            (x) float64 2kB 3.66e+05 3.66e+05 ... 3.681e+05 3.681e+05
    spatial_ref  int32 4B 32635
    time         datetime64[ns] 8B 2019-12-01T09:44:50.584000
    band         <U7 28B 'coastal'
Attributes:
    crs:      PROJCRS["WGS 84 / UTM zone 35N",BASEGEOGCRS["WGS 84",ENSEMBLE["...

In [17]:
rgb = (grass_land_da.sel(band=["red", "green", "blue"]) / 1000).clip(0,1)
rgb_plot = rgb.hvplot.rgb(
    x="x", y="y", bands="band", xlabel="lon", ylabel="lat", title="Sentinel-2 RGB",
)

cluster_plot = clusters.hvplot(
    x="x", y="y", cmap="Set3", xlabel="lon", ylabel="lat", colorbar=False, title="Spectral Clusters",
)

rgb_plot + cluster_plot


:Layout
   .RGB.I   :RGB   [x,y]   (R,G,B)
   .Image.I :Image   [x,y]   (value)

In [18]:
grass_lon = 3.666e+5
grass_lat = 6.213e+6

proj = pyproj.Proj(grass_land_da.crs)
grass_x, grass_y = proj(grass_lon, grass_lat)

grass = int(clusters.sel(x=grass_lon, y=grass_lat, method='nearest').values)

grass
# print('water cluster values:', water_cluster_1988, water_cluster_2017)

1

In [19]:
with xr.set_options(keep_attrs=True):
    grass_cluster = (clusters == grass).astype(int)

grass_cluster.hvplot()

:Image   [x,y]   (value)

In [22]:
mask = grass_cluster.fillna(0).astype('uint8')
mask_arr = mask.data
transform = grass_cluster.odc.geobox.transform
crs = grass_cluster.odc.crs

polygons = [
    shape(geom)
    for geom, value in features.shapes(mask_arr, mask=mask_arr == 1, transform=transform)
    if value == 1
]

grass_polygons = gpd.GeoDataFrame({'geometry': polygons}, crs=crs)
largest_geom = max(polygons, key=lambda geom: geom.area) if polygons else None
grass_largest = (
    gpd.GeoDataFrame({'geometry': [largest_geom]}, crs=crs)
    if largest_geom is not None
    else None
)
grass_largest.explore()


In [23]:
grass_largest.area

0    96400.0
dtype: float64